# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'VJR188R29R41TM17',
  'HostId': 's4FHW/aZc5kky61TAWMIXkCIL3D91tXKV3ZjLxcofHQF/LM/1wDZP71ZLSc5E8Z/P+T7CEaY29k=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 's4FHW/aZc5kky61TAWMIXkCIL3D91tXKV3ZjLxcofHQF/LM/1wDZP71ZLSc5E8Z/P+T7CEaY29k=',
   'x-amz-request-id': 'VJR188R29R41TM17',
   'date': 'Sat, 15 Mar 2025 23:48:50 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'era5-bra

In [6]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-03-15 18:50:51,484][INFO    ][coiled] Fetching latest package priorities...
[2025-03-15 18:50:51,485][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-03-15 18:50:52,217][INFO    ][coiled.package_sync] Scanning 442 conda packages...
[2025-03-15 18:50:52,225][INFO    ][coiled.package_sync] Scanning 258 python packages...
[2025-03-15 18:50:52,842][INFO    ][coiled] Running pip check...
[2025-03-15 18:50:54,194][INFO    ][coiled] Validating environment...
[2025-03-15 18:50:57,965][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-03-15 18:50:57,972][INFO    ][coiled] Creating wheel for /opt/spark-2.2.0/python...
[2025-03-15 18:50:57,991][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-03-15 18:50:57,992][INFO    ][coiled] Uploading coiled

ClusterCreationError: Cluster status is error (reason: Scheduler Stopped -> Software build failed -> Pip packages in your software environment could not be installed.

Error message:

Using Python 3.12.9 environment at env
Resolved 41 packages in 1.68s
  × Failed to download and build `pyspark==2.2.3`
  ╰─▶ Build backend failed to build wheel through `build_wheel` (exit status:
      1)

      [stderr]
      Traceback (most recent call last):
        File "<string>", line 11, in <module>
        File
      "/opt/coiled/cache/uv/builds-v0/.tmprucIyb/lib/python3.12/site-packages/setuptools/build_meta.py",
      line 435, in build_wheel
          return _build(['bdist_wheel'])
                 ^^^^^^^^^^^^^^^^^^^^^^^
        File
      "/opt/coiled/cache/uv/builds-v0/.tmprucIyb/lib/python3.12/site-packages/setuptools/build_meta.py",
      line 426, in _build
          return self._build_with_temp_dir(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
        File
      "/opt/coiled/cache/uv/bui

# RUN

In [5]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]

In [6]:
len(cities)

289

In [ ]:
import dask_geopandas as dgpd
from dask import delayed, compute, visualize
from dask.diagnostics import ProgressBar
%autoreload
from citywide_calculation import get_utm_crs
from metrics_calculation import calculate_minimum_distance_to_roads_option_B
from shapely.geometry import MultiLineString, LineString

@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg

@delayed
def load_dataset(path, epsg=None):
    """Load a single parquet dataset"""
    dataset = dgpd.read_parquet(path, npartitions=2)
    if epsg:
        dataset = dataset.set_crs("EPSG:4326", allow_override=True) 
        dataset = dataset.to_crs(epsg=epsg)
    return dataset

@delayed
def row_count(dgdf):
    """Count the rows in a dataframe"""
    row_count = dgdf.map_partitions(len).compute().sum()

    return row_count


def test_math(input):
    return input + input

%autoreload
from metrics_groupby import metrics

@delayed
def metrics(city_name,YOUR_NAME,grid_size):
    grid_cell_count = 0
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_search_buffer.geoparquet', #{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }
    # Get EPSG
    epsg = get_epsg(city_name)
    # Load grid
    grid = load_dataset(paths['grid'], epsg=epsg).compute()
    grid['cell_area'] = grid.geometry.area

    cells = grid.index.size
    grid_cell_count += cells.compute()

    # Load buildings and perform relevant calculations on it
    buildings = load_dataset(paths['buildings'], epsg=epsg).compute()
    buildings['area'] = buildings.geometry.area
    joined_buildings = dgpd.sjoin(buildings, grid, predicate='within')  
    counts_buildings = joined_buildings.groupby('index_right').size()
    grid['n_buildings'] = grid.index.map(counts_buildings).fillna(0).astype(int)
    built_area_buildings = joined_buildings.groupby('index_right')['area'].sum()
    grid['built_area'] = grid.index.map(built_area_buildings).fillna(0).astype(float)

    #total_buildings = row_count(buildings).compute()
    #print(total_buildings)
    # Load roads
    #roads = load_dataset(paths['roads'], epsg=epsg).compute()
    #road_union = roads.unary_union.compute()
    # Load intersections
    intersections = load_dataset(paths['intersections'], epsg=epsg).compute()
    print(type(intersections))
    intersections_3plus = intersections[intersections.street_count >= 3]
    print(type(intersections_3plus))
    intersections_4way = intersections[intersections.street_count == 4]
    print(type(intersections_4way))

    '''
    buildings['distance_to_road'] = buildings['geometry'].map_partitions(
        lambda partition: partition.apply(lambda geom: calculate_minimum_distance_to_roads_option_B(geom, road_union)),
        meta=('distance_to_road', 'float64')
    )
    
    buildings_within_10m_of_buildings = buildings[buildings.distance_to_road <= 20]
    joined_buildings_within_10m_of_buildings = dgpd.sjoin(buildings_within_10m_of_buildings, grid, predicate='within')
    counts_buildings_within_10m_of_buildings = joined_buildings_within_10m_of_buildings.groupby('index_right').size()
    grid['n_buildings_within_10m_of_roads'] = grid.index.map(counts_buildings_within_10m_of_buildings).fillna(0).astype(int)
    joined_buildings_distance_to_road = dgpd.sjoin(buildings[['geometry', 'distance_to_road']], grid, predicate='within')
    # NOT SURE OF THIS YET
    averaged_buildings_distance_to_road = joined_buildings_distance_to_road.groupby('index_right')['distance_to_road'].mean()
    grid['average_distance_to_closest_roads'] = grid.index.map(averaged_buildings_distance_to_road).fillna(0).astype(int)
    '''
    joined_intersections_3plus = dgpd.sjoin(intersections_3plus, grid, predicate='within')
    counts_intersections_3plus = joined_intersections_3plus.groupby('index_right').size()
    grid['intersections_3plus'] = grid.index.map(counts_intersections_3plus).fillna(0).astype(int)

    joined_intersections_4way = dgpd.sjoin(intersections_4way, grid, predicate='within')
    counts_intersections_4way = joined_intersections_4way.groupby('index_right').size()
    grid['intersections_4way'] = grid.index.map(counts_intersections_4way).fillna(0).astype(int)
    
    '''
    grid['m1'] = grid['n_buildings_within_10m_of_roads'] / grid['n_buildings']
    grid['m2'] = grid['average_distance_to_closest_roads']
    '''

    grid['m4'] = grid['intersections_4way'] / grid['intersections_3plus']


    grid['m11'] = 1.0*grid['n_buildings'] / grid['cell_area'] # Building density
    grid['m12'] = grid['built_area'] / grid['cell_area'] # Built area share
    grid['m13'] = grid['built_area'] / grid['n_buildings'] # Average building area

    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}.geoparquet'
    grid.to_parquet(path)
    return grid_cell_count, path


In [10]:
metrics('Accra', YOUR_NAME, grid_size=grid_size).compute()

(1,
 's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Accra/Accra_200m_grid_sara.geoparquet')

In [11]:
print(cities)

# Create delayed tasks for counting
grid_calculations = []

for city_name in cities:
    grid_calc = metrics(city_name, YOUR_NAME, grid_size=grid_size)
    grid_calculations.append(grid_calc)

#visualize(*grid_calulations)
calculated_grids = compute(*grid_calculations)
calculated_grids

['Abeokuta', 'Abidjan', 'Abuja', 'Accra', 'Addis_Ababa', 'Aden', 'Agra', 'Agri', 'Ahmadabad', 'Aleppo', 'Alexandria', 'Algiers', 'Aligarh', 'Alor_Star', 'Amman', 'Amravati', 'Ankara', 'Annaba', 'Anqing', 'Antalya', 'Antananarivo', 'Anyang', 'Arak', 'Arusha', 'Asuncion', 'Bac_Lieu', 'Bacolod', 'Baghdad', 'Baku', 'Bamako', 'Bamenda', 'Bandar_Lampung', 'Bandung', 'Bangkok', 'Bangui', 'Banjarbaru', 'Bankura', 'Barquisimeto', 'Barranquilla', 'Barreiras', 'Barretos', 'Bauchi', 'Beijing', 'Beira', 'Beirut', 'Belem', 'Belo_Horizonte', 'Benxi', 'Bhimavaram', 'Bhopal', 'Bicheng', 'Bilaspur', 'Bilbeis', 'Bogor', 'Bogota', 'Bou_Saada', 'Bouake', 'Buenos_Aires', 'Bukavu', 'Bukhara', 'Buraydah', 'Burewala', 'Bursa', 'Busan', 'Cabimas', 'Cabo_Frio', 'Cairo', 'Cali', 'Campinas', 'Can_Tho', 'Cape_Town', 'Caracas', 'Cartagena', 'Casablanca', 'Cebu_City', 'Changzhi', 'Changzhou', 'Chengde', 'Chengdu', 'Chennai', 'Chenzhou', 'Chittoor', 'Chongqing', 'Cirebon', 'Cochabamba', 'Coimbatore', 'Conakry', 'Const

((1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Abeokuta/Abeokuta_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Abidjan/Abidjan_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Abuja/Abuja_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Accra/Accra_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Addis_Ababa/Addis_Ababa_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Aden/Aden_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Agra/Agra_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Agri/Agri_200m_grid_sara.geoparquet'),
 (1,
  's3://wri-cities-

In [ ]:
# Sum the total number of grid cells
total_grid_cells = sum([grid_cells for grid_cells, path in calculated_grids])
print(f'Total grid cells: {total_grid_cells}')

Total grid cells: 462160


In [10]:
client.close()